In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

In [97]:
def vcf_to_df(Run_folder,sname):
    with open(f"{Run_folder}/{sname}") as f:
         li = f.read().rstrip().split('\n')
    flag=0
    row_li=[];vcf_header=[]
    for line in li:
        if flag:
            row = line.split('\t')
            row_li.append(row)
        if line.startswith('#CHROM'):
            head = line.replace('#','').split('\t')
            flag=1
        if line.startswith('##'):
            vcf_header.append(line)

    df_vcf = pd.DataFrame(row_li,columns=head)
    header = [info.split('=')[0] for info in df_vcf.INFO[0].split(';')]

    content=[]
    for i in range(len(df_vcf)):
        content.append([info.split('=')[1] for info in df_vcf.INFO[i].split(';')])

    df_info = pd.DataFrame(content,columns=header)
    df_info.AF = df_info.AF.astype(float)
    return df_vcf,vcf_header,df_info

def process_df_vcf(df_vcf,format_li):

    idx_li=[]
    for feat in format_li:
        globals()[f"{feat}_idx"] = df_vcf.FORMAT[0].split(':').index(feat)
        idx_li.append(globals()[f"{feat}_idx"])

    for col in ['DP','RD','AD','AF']:
        globals()[f"{col}_li"]=[]

    arr_li = [np.array(row.split(':'))[idx_li] for row in df_vcf.iloc[:,-1]]

    for arr in arr_li:
        DP_li.append(int(arr[0]))
        RD_li.append(int(arr[1].split(',')[0]))
        AD_li.append(int(arr[1].split(',')[1]))
        AF_li.append(float(arr[2]))

    df_vcf['DP'] = DP_li
    df_vcf['RD'] = RD_li
    df_vcf['AD'] = AD_li
    df_vcf['AF'] = AF_li
    return df_vcf[['CHROM', 'POS', 'ID', 'REF', 'ALT', 'FILTER','DP','RD','AD','AF']]


In [116]:
Run_folder='SNU_VIP'
sname_li=['LAH053','LAH052']
file_li = [sname+'.raw.vcf' for sname in sname_li]

In [117]:
format_li = ['DP','AD','AF']

In [118]:
cfDNA,header1,cfDNA_info = vcf_to_df(Run_folder, file_li[0])
cfDNA = process_df_vcf(cfDNA,format_li)

PBMC,header2,PBMC_info = vcf_to_df(Run_folder, file_li[1])
PBMC = process_df_vcf(PBMC,format_li)

In [121]:
cfDNA[cfDNA.FILTER=='PASS']

,CHROM,POS,ID,REF,ALT,FILTER,DP,RD,AD,AF
32,chr1,11115465,.,G,A,PASS,1905,1901,3,0.0016
34,chr1,11121270,.,C,T,PASS,1727,923,783,0.4534
92,chr1,11130589,.,G,A,PASS,2612,1280,1294,0.4954
135,chr1,11145001,.,C,T,PASS,2371,1217,1117,0.4711
210,chr1,11228701,.,G,A,PASS,2748,1394,1322,0.4811
...,...,...,...,...,...,...,...,...,...,...
20487,chrX,67545316,.,T,A,PASS,382,360,3,0.0079
20488,chrX,67545400,.,GGCA,G,PASS,351,346,4,0.0114
20536,chrX,67721838,.,G,A,PASS,1154,1150,3,0.0026
20545,chrX,101353203,.,G,A,PASS,777,0,777,1.0000


In [124]:
df_merge = pd.merge(cfDNA,PBMC,on=['CHROM', 'POS','ID', 'REF', 'ALT'],how='left',suffixes=['_c','_p'])

In [125]:
df_merge

,CHROM,POS,ID,REF,ALT,FILTER_c,DP_c,RD_c,AD_c,AF_c,FILTER_p,DP_p,RD_p,AD_p,AF_p
0,chr1,11107484,.,GT,G,T1;diffdepth;lowVD;lowpower,450,448,1,0.0022,NaN,NaN,NaN,NaN,NaN
1,chr1,11108185,.,C,A,T1;lowFreq001;lowVD;lowpower,1896,1895,1,0.0005,NaN,NaN,NaN,NaN,NaN
2,chr1,11108206,.,T,C,T1;lowFreq001;lowVD;lowpower,2115,2113,1,0.0005,NaN,NaN,NaN,NaN,NaN
3,chr1,11108251,.,G,A,T1;lowFreq001;lowVD,2260,2258,2,0.0009,NaN,NaN,NaN,NaN,NaN
4,chr1,11108257,.,C,T,T1;lowFreq001;lowVD,2319,2316,2,0.0009,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20646,chrX,124090615,.,A,G,T1;lowVD;lowpower,647,646,1,0.0015,NaN,NaN,NaN,NaN,NaN
20647,chrX,124090723,.,TC,T,T1;lowFreq001;lowVD;lowpower,1126,1125,1,0.0009,NaN,NaN,NaN,NaN,NaN
20648,chrX,124090730,.,C,T,T1;lowVD,1134,1132,2,0.0018,NaN,NaN,NaN,NaN,NaN
20649,chrX,124090929,.,C,T,T1;lowFreq001;lowVD;lowpower,1422,1421,1,0.0007,NaN,NaN,NaN,NaN,NaN


In [114]:
Multi_idx = ['cfDNA_LAH053','PBMC_LAH052']

mv=0;column=[('Common',col) for col in df_merge.columns[:5]]

for i in range(2):
    col_li = ['FILTER','DP','RD','AD','AF']
    mv+=5
    column.extend([(Multi_idx[i],col) for col in col_li])

df_merge2 = df_merge.copy()

df_merge2.columns=pd.MultiIndex.from_tuples(column)

In [115]:
df_merge2.fillna('-').to_excel('cfDNA_LAH053_VS_PBMC_LAH052_PASS.xlsx')

In [129]:
df_merge2

Common                         cfDNA_LAH053                            \
     CHROM        POS ID   REF ALT       FILTER    DP    RD    AD      AF   
0    chr1   11115465   .  G     A   PASS         1905  1901  3     0.0016   
1    chr1   11121270   .  C     T   PASS         1727  923   783   0.4534   
2    chr1   11130589   .  G     A   PASS         2612  1280  1294  0.4954   
3    chr1   11145001   .  C     T   PASS         2371  1217  1117  0.4711   
4    chr1   11228701   .  G     A   PASS         2748  1394  1322  0.4811   
..    ...        ...  .. ..    ..    ...          ...   ...   ...     ...   
611  chrX   67545316   .  T     A   PASS         382   360   3     0.0079   
612  chrX   67545400   .  GGCA  G   PASS         351   346   4     0.0114   
613  chrX   67721838   .  G     A   PASS         1154  1150  3     0.0026   
614  chrX   101353203  .  G     A   PASS         777   0     777   1.0000   
615  chrX   124071266  .  C     A   PASS         1180  1176  3     0.0025   

    PBMC_LAH052                               
         FILTER     DP     RD     AD      AF  
0    NaN        NaN    NaN    NaN    NaN      
1    PASS        392.0  225.0  167.0  0.4260  
2    PASS        321.0  174.0  146.0  0.4548  
3    PASS        461.0  234.0  225.0  0.4881  
4    PASS        778.0  384.0  393.0  0.5051  
..    ...          ...    ...    ...     ...  
611  NaN        NaN    NaN    NaN    NaN      
612  NaN        NaN    NaN    NaN    NaN      
613  NaN        NaN    NaN    NaN    NaN      
614  PASS        240.0  0.0    240.0  1.0000  
615  NaN        NaN    NaN    NaN    NaN      

[616 rows x 15 columns]

In [159]:
df_merge.query('AF_c > 0.5')

,CHROM,POS,ID,REF,ALT,FILTER_c,DP_c,RD_c,AD_c,AF_c,FILTER_p,DP_p,RD_p,AD_p,AF_p
1067,chr1,155910782,.,C,G,PASS,2105,0,2105,1.0000,PASS,603.0,0.0,603.0,1.0000
1774,chr2,29193500,.,G,C,PASS,2332,0,2332,1.0000,PASS,458.0,0.0,457.0,0.9978
1780,chr2,29193615,.,T,C,PASS,2345,0,2342,0.9987,PASS,235.0,0.0,235.0,1.0000
1790,chr2,29193706,.,T,C,PASS,2470,0,2466,0.9984,PASS,315.0,0.0,315.0,1.0000
1860,chr2,29222592,.,G,T,PASS,2389,2,2384,0.9979,PASS,774.0,15.0,755.0,0.9755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19820,chr20,58903752,.,C,T,PASS,2256,0,2255,0.9996,PASS,322.0,0.0,322.0,1.0000
20045,chr22,23290146,.,G,A,PASS,1602,0,1602,1.0000,PASS,317.0,0.0,317.0,1.0000
20106,chr22,23291065,.,G,A,PASS,1533,0,1530,0.9980,PASS,178.0,0.0,178.0,1.0000
20364,chrX,45069832,.,C,A,PASS,988,0,988,1.0000,PASS,141.0,0.0,141.0,1.0000


In [83]:
cfDNA2 = cfDNA[['CHROM', 'POS', 'REF', 'ALT','FILTER','FORMAT',sname_li[0]]].join(cfDNA_info[['TYPE', 'DP', 'VD', 'AF', 'QMean', 'MAPQ', 'NTUBE', 'POWER']])